#2021 Sigradi workshop
#Deep Architecture: Using AI in Architecture
##Instructor: Mohammed Behjoo
https://linktr.ee/MohammedBehjoo

###1. mounting GDrive and syncing it with Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###2. define path and directories to the folders

In [23]:
stlegan2_ada_path = "/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/colab-sg2-ada/stylegan2-ada" #@param {type:"string"}
network_snapshot_path = "/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/colab-sg2-ada/stylegan2-ada/results/my_images/00003-custom-mirror-stylegan2-kimg10000-ada-bgc-resumecustom/network-snapshot-000060.pkl" #@param {type:"string"}
result_path = "/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder"#@param {type:"string"}



%tensorflow_version 1.x
os.chdir(stlegan2_ada_path)

!nvidia-smi

import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy



if not os.path.exists(result_path):
    os.mkdir(result_path)
os.chdir(result_path)

Tue Nov  9 10:00:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    27W /  70W |   8286MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###3. Define Interpolation strategies

run this cell in order to initiazlie the interpolation approaches

In [12]:
!pip install opensimplex
"""Generate images using pretrained network pickle."""

import argparse
import sys
import os
import subprocess
import pickle
import re

import scipy
import numpy as np
import PIL.Image

import dnnlib
import dnnlib.tflib as tflib

os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
import moviepy.editor
from opensimplex import OpenSimplex

import warnings # mostly numpy warnings for me
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)


tflib.init_tf()
path = stlegan2_ada_path
pkl_path = network_snapshot_path

_G, _D, Gs = pickle.load(open(pkl_path, "rb"))
# _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
# _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
# Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.



############################################################

############################################################

def generate_images(seeds, truncation_psi):
    #_G, _D, Gs = pickle.load(open("/content/network-e621.pkl", "rb"))
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        #print(z)
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        PIL.Image.fromarray(images[0], 'RGB').save('seed%04d.png' % seed)
        #display(PIL.Image.fromarray(images[0], 'RGB'))


############################################################

############################################################


def truncation_traversal(network_pkl,outdir,class_idx=None, seed=[0],start=-1.0,stop=1.0,increment=0.1,framerate=24):
    tflib.init_tf()
    print('Loading networks from "%s"...' % network_pkl)
    #with dnnlib.util.open_url(network_pkl) as fp:
       # _G, _D, Gs = pickle.load(fp)

    os.makedirs(outdir, exist_ok=True)

    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    label = np.zeros([1] + Gs.input_shapes[1][1:])
    if class_idx is not None:
        label[:, class_idx] = 1

    count = 1
    trunc = start

    images = []
    while trunc <= stop:
        Gs_kwargs['truncation_psi'] = trunc
        print('Generating truncation psi %0.2f' % trunc)

        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        image = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
        images.append(image[0])
        #PIL.Image.fromarray(image[0], 'RGB').save(f'{outdir}/frame{count:05d}.png')

        trunc+=increment
        count+=1

    frames = moviepy.editor.ImageSequenceClip(images, fps=framerate)

            # Generate video at the current date and timestamp
    mp4_file = './trunc_trav_%s-%s-%s-%s.mp4'% (seed,increment,start,stop)
    mp4_codec = 'libx264'
    mp4_bitrate = '30M'
    mp4_fps = framerate # 20

    frames.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)


############################################################

############################################################


def blend_images(count,src_seed, dst_seed, blending=0.5, truncation_psi=0.7):
    #_G, _D, Gs = pickle.load(open("/content/network-e621.pkl", "rb"))

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    synthesis_kwargs = dict(output_transform=Gs_kwargs.output_transform, truncation_psi=truncation_psi, minibatch_size=8)
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi
    
    all_w = []

    for i, seed in enumerate([src_seed, dst_seed]):
        z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:])])
        #print(i, seed, z)
        all_w_src = Gs.components.mapping.run(z, None) # [minibatch, layer, component]
        if truncation_psi != 1:
            w_avg = Gs.get_var('dlatent_avg')
            all_w_src = w_avg + (all_w_src - w_avg) * truncation_psi # [minibatch, layer, component]
        all_w.append(all_w_src)
    
    w_new = (blending * all_w[0]) + (1 - blending) * all_w[1]
    images = Gs.components.synthesis.run(w_new, randomize_noise=False, **synthesis_kwargs)

    PIL.Image.fromarray(images[0], 'RGB').save(f"{count:04d}-" + f'{src_seed}-{dst_seed}-{blending}.png')



############################################################

############################################################



def valmap(value, istart, istop, ostart, ostop):
  return ostart + (ostop - ostart) * ((value - istart) / (istop - istart))

class OSN():
  min=-1
  max= 1

  def __init__(self,seed,diameter):
    self.tmp = OpenSimplex(seed)
    self.d = diameter
    self.x = 0
    self.y = 0

  def get_val(self,angle):
    self.xoff = valmap(np.cos(angle), -1, 1, self.x, self.x + self.d);
    self.yoff = valmap(np.sin(angle), -1, 1, self.y, self.y + self.d);
    return self.tmp.noise2d(self.xoff,self.yoff)

def get_noiseloop(endpoints, nf, d, start_seed):
    features = []
    zs = []
    for i in range(512):
      features.append(OSN(i+start_seed,d))

    inc = (np.pi*2)/nf
    for f in range(nf):
      z = np.random.randn(1, 512)
      for i in range(512):
        z[0,i] = features[i].get_val(inc*f)
      zs.append(z)

    return zs

def line_interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps)
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out

def generate_zs_from_seeds(seeds,Gs):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

def convertZtoW(latent, truncation_psi=0.7, truncation_cutoff=9):
    dlatent = Gs.components.mapping.run(latent, None) # [seed, layer, component]
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]
    dlatent = dlatent_avg + (dlatent - dlatent_avg) * truncation_psi

    return dlatent

def generate_latent_walk(network_pkl, truncation_psi, outdir, walk_type, frames, seeds, npys, save_vector, diameter=2.0, start_seed=0, framerate=24 ):
    global noise_vars
    tflib.init_tf()

    print('Loading networks from "%s"...' % network_pkl)
    #with dnnlib.util.open_url(network_pkl) as fp:
    #    _G, _D, Gs = pickle.load(fp)

    os.makedirs(outdir, exist_ok=True)
    os.makedirs(outdir+"/frames-"+ walk_type +'-'+f'{seeds}', exist_ok=True)
    if(save_vector):
        os.makedirs(outdir+"/vectors", exist_ok=True)

    # Render images for dlatents initialized from random seeds.
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    zs = []
    ws =[]


    # npys specified, let's work with these instead of seeds
    # npys must be saved as W's (arrays of 18x512)
    if npys and (len(npys) > 0):
        ws = npys


    wt = walk_type.split('-')

    if wt[0] == 'line':
        if seeds and (len(seeds) > 0):
            zs = generate_zs_from_seeds(seeds,Gs)

        if ws == []:
            number_of_steps = int(frames/(len(zs)-1))+1
        else:
            number_of_steps = int(frames/(len(ws)-1))+1

        if (len(wt)>1 and wt[1] == 'w'):
          if ws == []:
            for i in range(len(zs)):
              ws.append(convertZtoW(zs[i],truncation_psi))

          points = line_interpolate(ws,number_of_steps)
          # zpoints = line_interpolate(zs,number_of_steps)

        else:
          points = line_interpolate(zs,number_of_steps)


    # from Gene Kogan
    elif wt[0] == 'bspline':
        # bspline in w doesnt work yet
        # if (len(walk_type)>1 and walk_type[1] == 'w'):
        #   ws = []
        #   for i in range(len(zs)):
        #     ws.append(convertZtoW(zs[i]))

        #   print(ws[0].shape)
        #   w = []
        #   for i in range(len(ws)):
        #     w.append(np.asarray(ws[i]).reshape(512,18))
        #   points = get_latent_interpolation_bspline(ws,frames,3, 20, shuffle=False)
        # else:
          z = []
          for i in range(len(zs)):
            z.append(np.asarray(zs[i]).reshape(512))
          points = get_latent_interpolation_bspline(z,frames,3, 20, shuffle=False)

    # from Dan Shiffman: https://editor.p5js.org/dvs/sketches/Gb0xavYAR
    elif wt[0] == 'noiseloop':
        points = get_noiseloop(None,frames,diameter,start_seed)

    if (wt[0] == 'line' and len(wt)>1 and wt[1] == 'w'):
      # print(points[0][:,:,1])
      # print(zpoints[0][:,1])
      # ws = []
      # for i in enumerate(len(points)):
      #   ws.append(convertZtoW(points[i]))
        #added for npys
        if seeds:
            seed_out = 'w-' + wt[0] + ('-'.join([str(seed) for seed in seeds]))
        else:
            seed_out = 'w-' + wt[0] + '-dlatents'

        generate_images_in_w_space(points, truncation_psi,outdir,save_vector,'frame', seed_out, framerate)
    elif (len(wt)>1 and wt[1] == 'w'):
      print('%s is not currently supported in w space, please change your interpolation type' % (wt[0]))
    else:
        if(len(wt)>1):
            seed_out = 'z-' + wt[0] + ('-'.join([str(seed) for seed in seeds]))
        else:
            seed_out = 'z-' + walk_type + '-seed' +str(start_seed)
        generate_latent_images(points, truncation_psi, outdir, save_vector,'frame', seed_out, framerate)

def generate_latent_images(zs, truncation_psi, outdir, save_npy,prefix,vidname,framerate,class_idx=None):
    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    label = np.zeros([1] + Gs.input_shapes[1][1:])
    if class_idx is not None:
        label[:, class_idx] = 1

    for z_idx, z in enumerate(zs):
        if isinstance(z,list):
          z = np.array(z).reshape(1,512)
        elif isinstance(z,np.ndarray):
          z.reshape(1,512)
        print('Generating image for step %d/%d ...' % (z_idx, len(zs)))
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, label, **Gs_kwargs) # [minibatch, height, width, channel]
        PIL.Image.fromarray(images[0], 'RGB').save(f'{outdir}'+'/frames-'+ walk_type +'-'+f'{seeds}'+ '/'+ f'{prefix}{z_idx:05d}.png')
        if save_npy:
            np.save(f'{outdir}/vectors/{prefix}{z_idx:05d}.npz',z)
            # np.savetxt(f'{outdir}/vectors/{prefix}{z_idx:05d}.txt',z)
    #cmd="ffmpeg -y -r {} -i {}/frames/{}%05d.png -vcodec libx264 -pix_fmt yuv420p {}/walk-{}-{}fps.mp4".format(framerate,outdir+'/'+'frames-'+ walk_type +'-'+f'{seeds}',prefix,outdir,vidname,framerate)
    #subprocess.call(cmd, shell=True)

def generate_images_in_w_space(ws, truncation_psi,outdir,save_npy,prefix,vidname,framerate,class_idx=None):

    Gs_kwargs = {
        'output_transform': dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True),
        'randomize_noise': False
    }
    if truncation_psi is not None:
        Gs_kwargs['truncation_psi'] = truncation_psi

    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
    label = np.zeros([1] + Gs.input_shapes[1][1:])
    if class_idx is not None:
        label[:, class_idx] = 1

    for w_idx, w in enumerate(ws):
        print('Generating image for step %d/%d ...' % (w_idx, len(ws)))
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.components.synthesis.run(w, **Gs_kwargs) # [minibatch, height, width, channel]
        # images = Gs.run(w,label, **Gs_kwargs) # [minibatch, height, width, channel]
        PIL.Image.fromarray(images[0], 'RGB').save(f'{outdir}/frames/{prefix}{w_idx:05d}.png')
        if save_npy:
            np.save(f'{outdir}/vectors/{prefix}{w_idx:05d}.npz',w)
            # np.savetxt(f'{outdir}/vectors/{prefix}{w_idx:05d}.txt',w.reshape(w.shape[0], -1))

    #cmd="ffmpeg -y -r {} -i {}/frames/{}%05d.png -vcodec libx264 -pix_fmt yuv420p {}/walk-{}-{}fps.mp4".format(framerate,outdir+'/'+'frames-'+ walk_type +'-'+f'{seeds}',prefix,outdir,vidname,framerate)
    #subprocess.call(cmd, shell=True)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3457024/45929032 bytes (7.5%)7471104/45929032 bytes (16.3%)11083776/45929032 bytes (24.1%)14983168/45929032 bytes (32.6%)18866176/45929032 bytes (41.1%)22847488/45929032 bytes (49.7%)26804224/45929032 bytes (58.4%)30482432/45929032 bytes (66.4%)34160640/45929032 bytes (74.4%)38100992/45929032 bytes (83.0%)42147840/45929032 bytes (91.8%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

#Generating Images

###Generate Samples

In [ ]:
#@title Generate one image { form-width: "30%" }
%cd $result_path

seed = 107#@param{type:"integer"}
truncation_psi = 0 #@param {type:"slider", min:0, max:1, step:0.05}

generate_images([seed],truncation_psi)

/content/drive/MyDrive/dissertation/colab/output-folder/blend-two-seeds
Generating image for seed 107 (0/1) ...


In [15]:
#@title Generate multiple images { form-width: "30%" }

os.chdir(result_path)

import random

truncation_psi = 0.95 #@param {type:"slider", min:0, max:1, step:0.05}
start = 0 #@param{type:"integer"}
end = 500 #@param{type:"integer"}
numberOfSeed =  100#@param {type:"integer"}

#Generate 1 random numbers between defined start and end
randomlist = random.sample(range(start, end), numberOfSeed)
generate_images(randomlist,truncation_psi)

Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.
Generating image for seed 9 (0/100) ...
Generating image for seed 83 (1/100) ...
Generating image for seed 250 (2/100) ...
Generating image for seed 345 (3/100) ...
Generating image for seed 288 (4/100) ...
Generating image for seed 405 (5/100) ...
Generating image for seed 156 (6/100) ...
Generating image for seed 297 (7/100) ...
Generating image for seed 234 (8/100) ...
Generating image for seed 161 (9/100) ...
Generating image for seed 398 (10/100) ...
Generating image for seed 270 (11/100) ...
Generating image for seed 151 (12/100) ...
Generating image for seed 61 (13/100) ...
Generating image for seed 343 (14/100) ...
Generating image for seed 124 (15/100) ...
Generating image for seed 449 (16/100) ...
Generating image for seed 400 (17/100) ...
Generating image for seed 457 (18/100) ...
Generating image for seed 72 (19/100) ...

###Generate Truncation Traversal

In [18]:
#@title Generate truncation of the latent space { form-width: "30%" }
#Below you can take one seed and look at the changes to it across any truncation amount: -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets
#The Truncation Trick is a latent sampling procedure for generative adversarial networks, where we sample  from a truncated normal (where values which fall outside a range are resampled to fall inside that range).

#One of the challenges in generative models is dealing with areas that are poorly represented in the training data. 
#The generator isn’t able to learn them and create images that resemble them (and instead creates bad-looking images). 
#To avoid generating poor images, StyleGAN truncates the intermediate vector ⱳ, forcing it to stay close to the “average” 
#intermediate vector.

#After training the model, an “average” ⱳavg is produced by selecting many random inputs; generating their intermediate 
#vectors with the mapping network; and calculating the mean of these vectors. When generating new images, instead of 
#using Mapping Network output directly, ⱳ is transformed into ⱳ_new=ⱳ_avg+𝞧(ⱳ -ⱳ_avg), where the value of 𝞧 defines 
#how far the image can be from the “average” image (and how diverse the output can be). Interestingly, by using a 
#different 𝞧 for each level, before the affine transformation block, the model can control how far from average each 
#set of features is, as shown in the video below.


os.chdir(result_path)
seed = [1] #@param
#start trucation value
start = 0 #@param {type:"slider", min:-10, max:10, step:0.05}
#stopping truncation value. this should be larger than the start value
end = 1 #@param {type:"slider", min:-10, max:10, step:0.05}
#how much each frame should increment the truncation value. Set this small if you want a long,
increment = 0.01 #@param {type:"slider", min:0, max:0.5, step:0.01}

framerate = 30#@param{type:"slider", min:2, max:100, step:1}

truncation_traversal(Gs, path, None,seed=seed, start=start,stop=end, increment=increment,framerate=24)

/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/styleGAN2-output-interpolations
Loading networks from "<dnnlib.tflib.network.Network object at 0x7fe9328168d0>"...
Generating truncation psi 0.00
Generating truncation psi 0.01
Generating truncation psi 0.02
Generating truncation psi 0.03
Generating truncation psi 0.04
Generating truncation psi 0.05
Generating truncation psi 0.06
Generating truncation psi 0.07
Generating truncation psi 0.08
Generating truncation psi 0.09
Generating truncation psi 0.10
Generating truncation psi 0.11
Generating truncation psi 0.12
Generating truncation psi 0.13
Generating truncation psi 0.14
Generating truncation psi 0.15
Generating truncation psi 0.16
Generating truncation psi 0.17
Generating truncation psi 0.18
Generating truncation psi 0.19
Generating truncation psi 0.20
Generating truncation psi 0.21
Generating truncation psi 0.22
Generating truncation psi 0.23
Generating truncation psi 0.24
Generating truncation psi 0.25
Generating tr

100%|██████████| 100/100 [00:04<00:00, 21.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./trunc_trav_[1]-0.01-0-1.mp4 



###Generate Blending between two seeds with a fix blending factor

In [20]:
#@title Blend two seeds { form-width: "30%" }
os.chdir(result_path)

src_seed =  33#@param {type:"integer"}
dst_seed = 107 #@param {type:"integer"}
blending = 0.5 #@param {type:"slider", min:0, max:1, step:0.05}
truncation_psi = 1 #@param {type:"slider", min:0, max:1, step:0.05}
count=0
blend_images(count,src_seed, dst_seed, blending=blending, truncation_psi=truncation_psi)
print("Blended image (", blending, ")")

Blended image ( 0.5 )


###Generate Blending between two seeds with variable blending factor

In [22]:
#@title Blend two seeds { form-width: "30%" }
src_seed = 33 #@param {type:"raw"}
dst_seed = 107 #@param {type:"raw"}
blending_start = -1 #@param {type:"number", min:-5, max:5, step:0.01}
blending_end = 1 #@param {type:"number", min:-5, max:5, step:0.01}
blending_increment = 0.01 #@param {type:"slider", min:0, max:1, step:0.05}
truncation_psi = 0.95 #@param {type:"slider", min:0, max:1, step:0.05}

###---------------------------------------------------------------------------------------------------------------------------------
import decimal
def float_range(start, stop, step):
  while start < stop:
    yield float(start)
    start += decimal.Decimal(step)

print(list(float_range(blending_start, blending_end, blending_increment)))

#Create folder-------------------------------------------------------------------------------------------------------------------
outPath = result_path + '/' + f'{src_seed}-{dst_seed}' +'/'#'./%s-%s/' (src_seed,dst_seed)
print(outPath)
os.makedirs(outPath, exist_ok=True)
isFolder = os.path.isfile(outPath) 

###---------------------------------------------------------------------------------------------------------------------------------
if (isFolder != True):
  #print('File does not exist - ' + str(isFile))
  #print('Folder does not exist - ' + str(isFolder))
  %cd $outPath

  count = 0
  for i in float_range(blending_start,blending_end,blending_increment):
    count += 1
    print(str(i)+"-"+str(count))
    blend_images(count, src_seed, dst_seed, blending=i, truncation_psi=truncation_psi)

os.chdir(result_path)

[-1.0, -0.99, -0.98, -0.97, -0.96, -0.95, -0.94, -0.93, -0.92, -0.91, -0.9, -0.89, -0.88, -0.87, -0.86, -0.85, -0.84, -0.83, -0.82, -0.8099999999999999, -0.8, -0.79, -0.78, -0.77, -0.76, -0.75, -0.74, -0.73, -0.72, -0.71, -0.7, -0.69, -0.68, -0.67, -0.66, -0.65, -0.64, -0.63, -0.62, -0.61, -0.6, -0.59, -0.58, -0.57, -0.5599999999999999, -0.55, -0.54, -0.53, -0.52, -0.51, -0.5, -0.49, -0.48, -0.47, -0.45999999999999996, -0.45, -0.44, -0.43, -0.42, -0.41, -0.39999999999999997, -0.39, -0.38, -0.37, -0.36, -0.35, -0.33999999999999997, -0.32999999999999996, -0.32, -0.31, -0.3, -0.29, -0.27999999999999997, -0.26999999999999996, -0.26, -0.24999999999999997, -0.24, -0.22999999999999998, -0.21999999999999997, -0.21, -0.19999999999999998, -0.18999999999999997, -0.18, -0.16999999999999998, -0.15999999999999998, -0.15, -0.13999999999999999, -0.12999999999999998, -0.11999999999999998, -0.10999999999999999, -0.09999999999999998, -0.08999999999999998, -0.07999999999999999, -0.06999999999999998, -0.05

###Generate Interpolations between two seeds with variable blending factor

#####**Walk_Type**
NoiseLoop - allows us to generate a random interpolation of our model. It creates a random path thru the latent space to show us a diverse set of images.

diameter - controls how wide the noiseLoop is. The larger the diameter the larger the variety of samples.

start_seed - enables us to alter the latent space's starting point. This value is unrelated to the seeds used to generate the image. It merely randomizes our starting point.

In [27]:
#@title Interpolation { form-width: "30%" }


#to create rabdin interpolations of your model the noise loop is a great option... it creates a random path thru the z space to show you a diverse set of images

NeuralNetwork = _G #@param ["Gs", "_G", "_D"] {type:"raw", allow-input: true}

#use images you generated to control the interpolation points. If your first and last seed are the same this will produce a loop
seeds = [30302,79605,37989,984,6494,644] #@param

#allow the option to change the starting place in the z space (this value has nothing to do with the seed you use to generate images)
#but it allows you to randomize your start point
start_seed = 0 #@param

#truncation value
truncation_psi = 0.95 #@param {type:"slider", min:0, max:1, step:0.05}
walk_type = "line-z" #@param ["line-w", "line-z", "noiseloop"] {allow-input: false}

frames = 240 #@param {type:"integer", min:1, max:1000, step:1}
framerate = 24 #@param {type:"integer", min:1, max:100, step:1}

diameter = 198.7#@param{type:"slider", min:1, max:2000, step:0.1}

generate_latent_walk(NeuralNetwork, truncation_psi, result_path, walk_type, frames, seeds, npys= [], save_vector = False, diameter=diameter, start_seed=start_seed, framerate=framerate )

os.chdir(result_path)

Loading networks from "<dnnlib.tflib.network.Network object at 0x7fe99e5c4f10>"...
Generating image for step 0/245 ...
Generating image for step 1/245 ...
Generating image for step 2/245 ...
Generating image for step 3/245 ...
Generating image for step 4/245 ...
Generating image for step 5/245 ...
Generating image for step 6/245 ...
Generating image for step 7/245 ...
Generating image for step 8/245 ...
Generating image for step 9/245 ...
Generating image for step 10/245 ...
Generating image for step 11/245 ...
Generating image for step 12/245 ...
Generating image for step 13/245 ...
Generating image for step 14/245 ...
Generating image for step 15/245 ...
Generating image for step 16/245 ...
Generating image for step 17/245 ...
Generating image for step 18/245 ...
Generating image for step 19/245 ...
Generating image for step 20/245 ...
Generating image for step 21/245 ...
Generating image for step 22/245 ...
Generating image for step 23/245 ...
Generating image for step 24/245 ...
Ge

# creating videos out of frames

In [31]:
from PIL import Image, ImageFile
import os, sys, shutil
import cv2
import argparse


#@title createVideo
mainRoot = "/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder" #@param {type:"string"}

def frame2movie(dir_path,ext, videoName, output):
    images = []
    for f in os.listdir(dir_path):
        if f.endswith(ext):
            images.append(f)

    # Determine the width and height from the first image
    print(dir_path)
    image_path =  os.path.join(dir_path, images[0])
    frame = cv2.imread(image_path)
    #cv2.imshow('video',frame)
    height, width, channels = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
    out = cv2.VideoWriter(output + '/' + videoName, fourcc, 24.0, (width, height), True)

    for image in images:

        image_path = os.path.join(dir_path, image)
        frame = cv2.imread(image_path)
        print(image_path)
        out.write(frame) # Write out frame to video

        #cv2.imshow('video',frame)
        if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
            break

    # Release everything if job is finished
    out.release()
    cv2.destroyAllWindows()


for root, dirs, item in os.walk(mainRoot):
  #print("root_" + root)
  #print(dirs)
  #print(item) #pictures

  files = os.listdir( root )

  for d in range(0,len(dirs)):
    ext = "png" #@param {type:"string"}
    videoName = dirs[d] + '.mp4'
    print(videoName)

    dir_path = root + '/' + dirs[d]

    print('dir ---' + dir_path)
    print('output ---' + mainRoot)
    frame2movie(dir_path, ext, videoName, mainRoot)
    shutil.rmtree(dir_path)


frames-line-z-[30302, 79605, 37989, 984, 6494, 644].mp4
dir ---/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder/frames-line-z-[30302, 79605, 37989, 984, 6494, 644]
output ---/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder
/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder/frames-line-z-[30302, 79605, 37989, 984, 6494, 644]
/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder/frames-line-z-[30302, 79605, 37989, 984, 6494, 644]/frame00000.png
/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder/frames-line-z-[30302, 79605, 37989, 984, 6494, 644]/frame00001.png
/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder/frames-line-z-[30302, 79605, 37989, 984, 6494, 644]/frame00002.png
/content/drive/MyDrive/Deep_Architecture_using_AI_in_architecture/last_folder/frames-line-z-[30302, 79605, 37989, 984, 6494, 644]/frame00003.png
/content/